In [ ]:
import saspy
sas = saspy.SASsession()

### conversions - from date shaped char

In [ ]:
# date shaped char table
sas.submitLST(
f"""
data old_table;
    input date_column $;
    datalines;
'202101'
''
;
run;
""")

In [ ]:
def make_sql(key_str):
    return f"""
proc print data=old_table;
run;

proc sql;
    create table new_table as
    select
{key_str}
    from old_table;
quit;

proc print data=new_table;
run;
"""

In [ ]:
# date shaped char -> date
s = make_sql("""
        INPUT(CATS(date_column, '01'), YYMMDD10.) AS new_date FORMAT=DATE9.,
        INPUT(IFC(date_column = ' ', ' ', CATS(date_column, '01')), YYMMDD10.) AS new_date2 FORMAT=DATE9.
""")
sas.submitLST(s)

In [ ]:
# date shaped char -> date number
s = make_sql("""
        INPUT(new_date, 6.) AS date_number
""")
sas.submitLST(s)

### conversions - from date shaped num

In [ ]:
sas.submitLST(
f"""
data old_table;
    input date_column $;
    datalines;
202101
.
;
run;
""")

In [ ]:
# date shaped number -> date shaped char
s = make_sql("""
        PUT(date_column, 6.) AS new_date_char
""")
sas.submitLST(s)

### arithmetic of date shaped numbers

In [ ]:
sas.submitLST(
f"""
/* Add x number of months */
data add_months;
    input date $6.;
    year = substr(date,1,4);
    month = substr(date,5,2);
    x = 3; /* number of months to add */
    month = month + x;
    if month > 12 then do;
        year = year + 1;
        month = month - 12;
    end;
    new_date = catx('', year, put(month,z2.));
    format new_date $6.;
    datalines;
202409
;
run;
""")

In [ ]:
sas.submitLST(
f"""
/* Add y number of years */
data add_years;
    input date $6.;
    year = substr(date,1,4);
    month = substr(date,5,2);
    y = 2; /* number of years to add */
    year = year + y;
    new_date = catx('', year, month);
    format new_date $6.;
    datalines;
202409
;
run;
""")

In [ ]:
sas.submitLST(
f"""
/* Difference between two dates */
data date_diff;
    input date1 $6. date2 $6.;
    year1 = substr(date1,1,4);
    month1 = substr(date1,5,2);
    year2 = substr(date2,1,4);
    month2 = substr(date2,5,2);
    diff_years = year2 - year1;
    diff_months = month2 - month1;
    total_diff = diff_years*12 + diff_months;
    datalines;
202409 202512
;
run;
""")

In [ ]:
sas.submitLST(
f"""
/* Add x number of months */
proc sql;
    create table add_months as
    select date, intnx('month', input(date, yymmn6.), x) as new_date format=yymmn6.
    from (select '202409' as date)
    ;
quit;
""")

In [ ]:
sas.submitLST(
f"""
/* Add y number of years */
proc sql;
    create table add_years as
    select date, intnx('year', input(date, yymmn6.), y) as new_date format=yymmn6.
    from (select '202409' as date)
    ;
quit;
""")

In [ ]:
sas.submitLST(
f"""
/* Difference between two dates */
proc sql;
    create table date_diff as
    select date1, date2, intck('month', input(date1, yymmn6.), input(date2, yymmn6.)) as diff
    from (select '202409' as date1, '202512' as date2)
    ;
quit;
""")

### reference table of skeleton dates

### comparison to datestring

In [ ]:
sas.submitLST(f"""
    proc print data=sashelp.air;
        where date <= "01mar1949"d;
    run;
""")

Obs,DATE,AIR
1,JAN49,112
2,FEB49,118
3,MAR49,132


### input string as date

In [8]:
# forms that start with yy can be parsed via anydtdte10 informat.
sas.submitLST(f"""
    data df;
        input d1 anydtdte10.;
        format d1 ddmmyy10.;
        datalines;
2025-02-13
1993-02-13
1993/02/13
        ;
    run;
              
    proc print data=df;
    run;
""")

Obs,d1
1,13/02/2025
2,13/02/1993
3,13/02/1993


In [9]:
# a common form is ddmmyy10.
sas.submitLST(f"""
    data df;
        input d1 ddmmyy10.;
        format d1 ddmmyy10.;
        datalines;
13/02/2025
13-02-2025
13.02.2025
        ;
    run;
              
    proc print data=df;
    run;
""")

Obs,d1
1,13/02/2025
2,13/02/2025
3,13/02/2025


In [10]:
# the core date9 format is ddmmmyyyy
sas.submitLST(f"""
    data df;
        input d1 date9.;
        format d1 date9.;
        datalines;
13feb2025
13FEB2025
13feb40
        ;
    run;
              
    proc print data=df;
    run;
""")

sas.sasdata('df').columnInfo()

Obs,d1
1,13FEB2025
2,13FEB2025
3,13FEB1940


,Member,Num,Variable,Type,Len,Pos,Format
0,WORK.DF,1.0,d1,Num,8.0,0.0,DATE9.
